# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 3:00PM,243482,10,0085924472,ISDIN Corporation,Released
1,Aug 4 2022 3:00PM,243482,10,0085924471,ISDIN Corporation,Released
2,Aug 4 2022 3:00PM,243482,10,0085924479,ISDIN Corporation,Released
3,Aug 4 2022 3:00PM,243482,10,0085924460,ISDIN Corporation,Released
4,Aug 4 2022 3:00PM,243482,10,0085924459,ISDIN Corporation,Released
5,Aug 4 2022 3:00PM,243482,10,0085924470,ISDIN Corporation,Released
6,Aug 4 2022 3:00PM,243482,10,0085924473,ISDIN Corporation,Released
7,Aug 4 2022 3:00PM,243482,10,0085924474,ISDIN Corporation,Released
8,Aug 4 2022 3:00PM,243482,10,0085924475,ISDIN Corporation,Released
9,Aug 4 2022 3:00PM,243482,10,0085924449,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,243492,Released,1
26,243493,Released,2
27,243495,Released,3
28,243496,Released,5
29,243497,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243492,NaN,1.0
243493,NaN,2.0
243495,NaN,3.0
243496,NaN,5.0
243497,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243414,1.0,0.0
243436,0.0,1.0
243451,0.0,5.0
243455,0.0,1.0
243457,3.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243414,1,0
243436,0,1
243451,0,5
243455,0,1
243457,3,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243414,1,0
1,243436,0,1
2,243451,0,5
3,243455,0,1
4,243457,3,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243414,1,
1,243436,,1
2,243451,,5
3,243455,,1
4,243457,3,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation
23,Aug 4 2022 2:58PM,243497,12,Hush Hush
24,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc."
29,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC"
32,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC"
34,Aug 4 2022 2:46PM,243492,10,Emerginnova
35,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC"
36,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc."
38,Aug 4 2022 2:38PM,243489,10,"Methapharm, Inc."
52,Aug 4 2022 2:33PM,243488,10,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,,23
1,Aug 4 2022 2:58PM,243497,12,Hush Hush,,1
2,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",,5
3,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",,3
4,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",,2
5,Aug 4 2022 2:46PM,243492,10,Emerginnova,,1
6,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC",,1
7,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc.",,2
8,Aug 4 2022 2:38PM,243489,10,"Methapharm, Inc.",,14
9,Aug 4 2022 2:33PM,243488,10,"ACI Healthcare USA, Inc.",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23,
1,Aug 4 2022 2:58PM,243497,12,Hush Hush,1,
2,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5,
3,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",3,
4,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",2,
5,Aug 4 2022 2:46PM,243492,10,Emerginnova,1,
6,Aug 4 2022 2:40PM,243491,16,"SHL Pharma, LLC",1,
7,Aug 4 2022 2:38PM,243490,10,"Methapharm, Inc.",2,
8,Aug 4 2022 2:38PM,243489,10,"Methapharm, Inc.",14,
9,Aug 4 2022 2:33PM,243488,10,"ACI Healthcare USA, Inc.",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23,
1,Aug 4 2022 2:58PM,243497,12,Hush Hush,1,
2,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5,
3,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",3,
4,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23.0,NaN
1,Aug 4 2022 2:58PM,243497,12,Hush Hush,1.0,NaN
2,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5.0,NaN
3,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",3.0,NaN
4,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,23.0,0.0
1,Aug 4 2022 2:58PM,243497,12,Hush Hush,1.0,0.0
2,Aug 4 2022 2:56PM,243496,15,"Person & Covey, Inc.",5.0,0.0
3,Aug 4 2022 2:52PM,243495,15,"Mizner Bioscience, LLC",3.0,0.0
4,Aug 4 2022 2:51PM,243493,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3408670,106.0,0.0
12,730434,4.0,0.0
15,730484,10.0,0.0
16,243491,1.0,0.0
19,243451,5.0,0.0
20,973887,42.0,4.0
21,486897,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3408670,106.0,0.0
1,12,730434,4.0,0.0
2,15,730484,10.0,0.0
3,16,243491,1.0,0.0
4,19,243451,5.0,0.0
5,20,973887,42.0,4.0
6,21,486897,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,106.0,0.0
1,12,4.0,0.0
2,15,10.0,0.0
3,16,1.0,0.0
4,19,5.0,0.0
5,20,42.0,4.0
6,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,106.0
1,12,Released,4.0
2,15,Released,10.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,4.0,1.0
Released,106.0,4.0,10.0,1.0,5.0,42.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,4.0,1.0
1,Released,106.0,4.0,10.0,1.0,5.0,42.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,4.0,1.0
1,Released,106.0,4.0,10.0,1.0,5.0,42.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()